In [392]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib
import time

In [393]:
#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
driver = webdriver.Chrome('/Users/sunnie/Desktop/School/UIUC/CS410/MP2.1/chromedriver',options=options)

In [394]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,driver):
    driver.get(url)
    res_html = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

#tidies extracted text 
def process_bio(bio):
    bio = bio.encode('ascii',errors='ignore').decode('utf-8')       #removes non-ascii characters
    bio = re.sub('\s+',' ',bio)       #repalces repeated whitespace characters with single space
    return bio

''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup


#Checks if bio_url is a valid faculty homepage
def is_valid_homepage(bio_url,dir_url):
    if bio_url.endswith('.pdf'): #we're not parsing pdfs
        return False
    try:
        #sometimes the homepage url points to the same page as the faculty profile page
        #which should be treated differently from an actual homepage
        ret_url = urllib.request.urlopen(bio_url).geturl() 
    except:
        return False       #unable to access bio_url
    urls = [re.sub('((https?://)|(www.))','',url) for url in [ret_url,dir_url]] #removes url scheme (https,http or www) 
    return not(urls[0]== urls[1])

In [395]:
#extracts all Faculty Profile page urls from the Directory Listing Page
def scrape_dir_page(dir_url,driver):
    print ('-'*20,'Scraping directory page','-'*20)
    faculty_links = []
    faculty_base_url = 'https://www.sjsu.edu'
    #execute js on webpage to load faculty listings on webpage and get ready to parse the loaded HTML 
    soup = get_js_soup(dir_url,driver)    
    for link in soup.find_all('a', href=True):
        if "people" in link.get("href") or "edu/~" in link.get("href"):
            faculty_links.append(str(link.get('href')))
    print ('-'*20,'Found {} faculty profile urls'.format(len(faculty_links)),'-'*20)
    return faculty_links

In [396]:
dir_url = 'https://www.sjsu.edu/cs/faculty/faculty.php' #url of directory listings of CS faculty
faculty_links = scrape_dir_page(dir_url,driver)

-------------------- Scraping directory page --------------------
-------------------- Found 18 faculty profile urls --------------------


In [397]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)



def scrape_faculty_page(fac_url,driver):
    soup = get_js_soup(fac_url,driver)
    homepage_found = False
    bio_url = ''
    bio = ''
    profile_sec = soup.find('div', class_= 'content_wrapper')
#     if profile_sec is not None:
#         all_headers = profile_sec.find_all('titile')
#         faculty_last_name = all_headers[0].get_text().lower().split()[-1] #find faculty last name
#         faculty_first_name = all_headers[0].get_text().lower().split()[0]
#         homepage_txts = ['University',faculty_last_name,faculty_first_name]
#         #exceptions = ['course ','research','group','cs','mirror','google scholar']
#         #find the homepage url and extract all text from it
#         for hdr in all_headers:  #first find the required header
#             if hdr.text.lower() == 'for more information':
#                 next_tag = hdr.find_next('li')
#                 #find <li> which has homepage url
#                 while next_tag is not None: 
#                     cand = next_tag.find('a')
#                     next_tag = next_tag.next_sibling  #sibling means element present at the same level
#                     try:
#                         cand['href']
#                     except:
#                         continue
#                     cand_text = cand.string

#                     if cand_text is not None and (any(hp_txt in cand_text.lower() for hp_txt in homepage_txts) and 
#                         not any(e in cand_text.lower() for e in exceptions)): #compare text to predefined patterns
#                         bio_url = cand['href'] 
#                         homepage_found = True
#                         #check if homepage url is valid
#                         if not(is_valid_homepage(bio_url,fac_url)):
#                             homepage_found = False
#                         else:
#                             try:
#                                 bio_soup = remove_script(get_js_soup(bio_url,driver)) 
#                             except:
#                                 print ('Could not access {}'.format(bio_url))
#                                 homepage_found = False
#                         break 
#                 if homepage_found:
#                     #get all the text from homepage(bio)
#                     bio = process_bio(bio_soup.get_text(separator=' ')) 


#         if not homepage_found:
    bio_url = fac_url #treat faculty profile page as homepage
    
    if(soup.find('div', class_='content_wrapper')):
        profile_sec = soup.find('div', class_= 'content_wrapper')
        bio = process_bio(profile_sec.get_text(separator=' '))
    elif(soup.find('div', id_='content')):
        profile_sec = soup.find('div', id_= 'content')
        bio = process_bio(profile_sec.get_text(separator=' '))
    elif(soup.find('div', clss_='content_item')):
        profile_sec = soup.find('div', class_= 'content_item')
        bio = process_bio(profile_sec.get_text(separator=' '))    
    elif(soup.find_all("p")):
        for para in soup.find_all("p"):
            bio = para.get_text(separator=' ') 
    else:
        html = urllib.request.urlopen(bio_url).read()
        print(html)
        bio = text_from_html(html)
        print(bio)
    return bio_url,bio

In [398]:
#Scrape homepages of all urls
bio_urls, bios = [],[]
tot_urls = len(faculty_links)
for i,link in enumerate(faculty_links):
    print ('-'*20,'Scraping faculty url {}/{}'.format(i+1,tot_urls),'-'*20)
    bio_url,bio = scrape_faculty_page(link,driver)
    if bio.strip()!= '' and bio_url.strip()!='':
        bio_urls.append(bio_url.strip())
        bios.append(bio)
driver.close()

-------------------- Scraping faculty url 1/18 --------------------
-------------------- Scraping faculty url 2/18 --------------------
-------------------- Scraping faculty url 3/18 --------------------
-------------------- Scraping faculty url 4/18 --------------------
-------------------- Scraping faculty url 5/18 --------------------
-------------------- Scraping faculty url 6/18 --------------------
b'<HTML>\n<!--\n<HEAD>\n   <META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">\n   <META NAME="GENERATOR" CONTENT="Mozilla/4.05 [en] (X11; U; SunOS 5.5.1 sun4m) [Netscape]">\n   <TITLE> Dr. Suneuy Kim\'s Home Page </TITLE>\n</HEAD>\n<BODY TEXT="#000000" BGCOLOR="#FFFFCC" LINK="#0000FF" VLINK="#FF0000" ALINK="#0000FF">\n<TABLE BORDER=2 CELLSPACING=2 CELLPADDING=2 HEIGHT=30 WIDTH="80%">\n<TR ALIGN=BASELINE>\n<TD ALIGN=BASELINE WIDTH="12%" HEIGHT="20">\n<P><IMG SRC="./images/sekim.jpg" HEIGHT=180 WIDTH=140 ALIGN=TEXT TOP></P>\n</TD>\n<TD ALIGN=BASELINE HEIGHT="20" WID

In [399]:
def write_lst(lst,file_):
    with open(file_,'w') as f:
        for l in lst:
            f.write(l)
            f.write('\n')

In [400]:
bio_urls_file = 'bio_urls.txt'
bios_file = 'bios.txt'
write_lst(bio_urls,bio_urls_file)
write_lst(bios,bios_file)